CHANGELOG

v0.7 12/1/2023
- for data_test_v2

v0.6 11/15/2023
- for hold out and external testing

v0.0 6/16/23:
- BasicAug
- batches_all

v0.1 6/19/23
- added in pearson correlation

v0.2 6/19/23:
- added hausdorff distance
- added IoU
- added AUC ROC
- added AUC PR

v0.3 6/24/23
- added access to jerry_losses.py

v0.4 7/3/2023
- added in restart run all

v0.5 7/4/2023
- added in if and elif clauses to check for prior existing files before running things

In [1]:
from jarvis.utils.general import gpus
gpus.autoselect()

import glob, numpy as np, pandas as pd, tensorflow as tf, os, scipy, random
from tensorflow.keras import Input, Model, layers
from scipy.spatial.distance import directed_hausdorff
from IPython.display import clear_output, HTML, Javascript, display

import sys  
sys.path.append('/home/jjlou/Jerry/jerry_packages')
from jerry_utils import restart_kernel, load_dataset
import jerry_losses, jerry_metrics 

[ 2024-02-02 12:20:28 ] CUDA_VISIBLE_DEVICES automatically set to: 0           


In [2]:
name = 'Attention_Unfrozen_EfficientNetV2L_v0.72'
root = '/home/jjlou/Jerry/wsi-arterio/arteriosclerotic_vessel_detection_and_fine_segmentation/Vessel_WallsLumen_Segmentation/data_test_v2'

custom_objects = {
    'dice_all': jerry_metrics.dice_metric(cls=1),
    'hausdorff_all': jerry_metrics.hausdorff_metric(cls=1),
    'focal_dice_like_loss_multiclass_weighted': jerry_losses.focal_dice_like_loss_multiclass_weighted
}

In [3]:
def true_pred(true, pred, c):
    true = true.numpy()
    if c == None:
        true = true[..., 0]
        pred = np.argmax(pred, axis=-1)
    else:
        true = true[..., 0] == c
        pred = np.argmax(pred, axis=-1) == c
    return true, pred

In [4]:
calc_roc = tf.keras.metrics.AUC(curve='ROC')
calc_pr = tf.keras.metrics.AUC(curve='PR')
def AUC(y_true, y_pred, c=None):
    true, pred = true_pred(y_true, y_pred, c)
    calc_roc.update_state(true, pred)
    calc_pr.update_state(true, pred)
    roc = calc_roc.result().numpy()
    pr = calc_pr.result().numpy()
    calc_roc.reset_state()
    calc_pr.reset_state()
    return float(roc), float(pr)

In [5]:
m = tf.keras.metrics.MeanIoU(num_classes=3)
def mIoU(y_true, y_pred, c=None):
    true, pred = true_pred(y_true, y_pred, c)
    m.update_state(true, pred)
    r = m.result().numpy()
    m.reset_state()
    return float(r)

In [6]:
def hausdorff(y_true, y_pred, c=None):
    true, pred = true_pred(y_true, y_pred, c)
    true = np.squeeze(true)
    pred = np.squeeze(pred)
    d = directed_hausdorff(true, pred)[0]
    return d

In [7]:
def dice(y_true, y_pred, c=None):
    true, pred = true_pred(y_true, y_pred, c)
    true = true.astype('int64')
    A = np.count_nonzero(true & pred) * 2
    B = np.count_nonzero(true) + np.count_nonzero(pred)
    if np.count_nonzero(true) == 0 and np.count_nonzero(pred) == 0:
        A = 1
        B = 1
    return A / B

In [8]:
def pearson(y_true, y_pred, c=None):
    true, pred = true_pred(y_true, y_pred, c)
    true = true.flatten()
    pred = pred.flatten()
    pear, p = scipy.stats.pearsonr(pred,true)
    return pear, p

In [9]:
def vessel_lumen():
    # Vessel and lumen
    dsc_hold_vl = []
    hold_vl_pearson = []
    hold_vl_p = []
    hold_vl_hausdorff = []
    hold_vl_mIoU = []
    hold_vl_ROC = []
    hold_vl_PR = []
    for x,y in hold:
        logits = model.predict(x) 
        if type(logits) is dict:
            logits = logits['r1']
        dsc_hold_vl.append(dice(y, logits)) # dice
        hold_vl_hausdorff.append(hausdorff(y, logits)) # hausdorff
        hold_vl_mIoU.append(mIoU(y, logits)) #mIoU
        roc, pr = AUC(y, logits) #AUC ROC and PR
        hold_vl_ROC.append(roc)
        hold_vl_PR.append(pr)
        pear, p = pearson(y, logits) # pearson
        hold_vl_pearson.append(pear)
        hold_vl_p.append(p) 

    dsc_external_vl = []
    external_vl_pearson = []
    external_vl_p = []
    external_vl_hausdorff = []
    external_vl_mIoU = []
    external_vl_ROC = []
    external_vl_PR = []
    for x,y in external:
        logits = model.predict(x) 
        if type(logits) is dict:
            logits = logits['r1']
        dsc_external_vl.append(dice(y, logits))
        external_vl_hausdorff.append(hausdorff(y, logits))
        external_vl_mIoU.append(mIoU(y, logits))
        roc, pr = AUC(y, logits)
        external_vl_ROC.append(roc)
        external_vl_PR.append(pr)
        pear, p = pearson(y, logits)
        external_vl_pearson.append(pear)
        external_vl_p.append(p)            

    df_hold_vl = pd.DataFrame(index=np.arange(len(dsc_hold_vl)))
    df_external_vl = pd.DataFrame(index=np.arange(len(dsc_external_vl)))
    df_hold_vl['Hold dice score'] = dsc_hold_vl
    df_external_vl['External dice score'] = dsc_external_vl
    df_hold_vl['Hold pearson'] = hold_vl_pearson
    df_hold_vl['Hold p'] = hold_vl_p
    df_external_vl['External pearson'] = external_vl_pearson
    df_external_vl['External p'] = external_vl_p
    df_hold_vl['Hold hausdorff'] = hold_vl_hausdorff
    df_external_vl['External hausdorff'] = external_vl_hausdorff
    df_hold_vl['Hold mIoU'] = hold_vl_mIoU
    df_external_vl['External mIoU'] = external_vl_mIoU
    df_hold_vl['Hold ROC'] = hold_vl_ROC
    df_hold_vl['Hold PR'] = hold_vl_PR
    df_external_vl['External ROC'] = external_vl_ROC
    df_external_vl['External PR'] = external_vl_PR
    df_hold_vl.to_csv(f'{save_path}_hold_vl.csv')
    df_external_vl.to_csv(f'{save_path}_external_vl.csv')

In [10]:
def vessel():
   # Vessel
    dsc_hold_v = []
    hold_v_pearson = []
    hold_v_p = []
    hold_v_hausdorff = []
    hold_v_mIoU = []
    hold_v_ROC = []
    hold_v_PR = []
    for x,y in hold:
        logits = model.predict(x) 
        if type(logits) is dict:
            logits = logits['r1']
        dsc_hold_v.append(dice(y, logits,c=1))
        hold_v_hausdorff.append(hausdorff(y, logits, c=1))
        hold_v_mIoU.append(mIoU(y, logits, c=1))
        roc, pr = AUC(y, logits, c=1) 
        hold_v_ROC.append(roc)
        hold_v_PR.append(pr)
        pear, p = pearson(y, logits, c=1)
        hold_v_pearson.append(pear)
        hold_v_p.append(p) 

    dsc_external_v = []
    external_v_pearson = []
    external_v_p = []
    external_v_hausdorff = []
    external_v_mIoU = []
    external_v_ROC = []
    external_v_PR = []
    for x,y in external:
        logits = model.predict(x) 
        if type(logits) is dict:
            logits = logits['r1']
        dsc_external_v.append(dice(y, logits,c=1))
        external_v_hausdorff.append(hausdorff(y, logits, c=1))
        external_v_mIoU.append(mIoU(y, logits, c=1))
        roc, pr = AUC(y, logits, c=1) 
        external_v_ROC.append(roc)
        external_v_PR.append(pr)
        pear, p = pearson(y, logits, c=1)
        external_v_pearson.append(pear)
        external_v_p.append(p) 

    df_hold_v = pd.DataFrame(index=np.arange(len(dsc_hold_v)))
    df_external_v = pd.DataFrame(index=np.arange(len(dsc_external_v)))
    df_hold_v['Hold dice score'] = dsc_hold_v
    df_external_v['External dice score'] = dsc_external_v
    df_hold_v['Hold pearson'] = hold_v_pearson
    df_hold_v['Hold p'] = hold_v_p
    df_external_v['External pearson'] = external_v_pearson
    df_external_v['External p'] = external_v_p
    df_hold_v['Hold hausdorff'] = hold_v_hausdorff
    df_external_v['External hausdorff'] = external_v_hausdorff
    df_hold_v['Hold mIoU'] = hold_v_mIoU
    df_external_v['External mIoU'] = external_v_mIoU
    df_hold_v['Hold ROC'] = hold_v_ROC
    df_hold_v['Hold PR'] = hold_v_PR
    df_external_v['External ROC'] = external_v_ROC
    df_external_v['External PR'] = external_v_PR
    df_hold_v.to_csv(f'{save_path}_hold_v.csv')
    df_external_v.to_csv(f'{save_path}_external_v.csv')

In [11]:
def lumen():
    # Lumen
    dsc_hold_l = []
    hold_l_pearson = []
    hold_l_p = []
    hold_l_hausdorff = []
    hold_l_mIoU = []
    hold_l_ROC = []
    hold_l_PR = []
    for x,y in hold:
        logits = model.predict(x) 
        if type(logits) is dict:
            logits = logits['r1']
        dsc_hold_l.append(dice(y, logits,c=2))
        hold_l_hausdorff.append(hausdorff(y, logits, c=2))
        hold_l_mIoU.append(mIoU(y, logits, c=2))
        roc, pr = AUC(y, logits, c=2) 
        hold_l_ROC.append(roc)
        hold_l_PR.append(pr)
        pear, p = pearson(y, logits, c=2)
        hold_l_pearson.append(pear)
        hold_l_p.append(p) 

    dsc_external_l = []
    external_l_pearson = []
    external_l_p = []
    external_l_hausdorff = []
    external_l_mIoU = []
    external_l_ROC = []
    external_l_PR = []
    for x,y in external:
        logits = model.predict(x) 
        if type(logits) is dict:
            logits = logits['r1']
        dsc_external_l.append(dice(y, logits,c=2))
        external_l_hausdorff.append(hausdorff(y, logits, c=2))
        external_l_mIoU.append(mIoU(y, logits, c=2))
        roc, pr = AUC(y, logits, c=2) 
        external_l_ROC.append(roc)
        external_l_PR.append(pr)
        pear, p = pearson(y, logits, c=2)
        external_l_pearson.append(pear)
        external_l_p.append(p) 

    df_hold_l = pd.DataFrame(index=np.arange(len(dsc_hold_l)))
    df_external_l = pd.DataFrame(index=np.arange(len(dsc_external_l)))
    df_hold_l['Hold dice score'] = dsc_hold_l
    df_external_l['External dice score'] = dsc_external_l
    df_hold_l['Hold pearson'] = hold_l_pearson
    df_hold_l['Hold p'] = hold_l_p
    df_external_l['External pearson'] = external_l_pears
    df_external_l['External p'] = external_l_p
    df_hold_l['Hold hausdorff'] = hold_l_hausdorff
    df_external_l['External hausdorff'] = external_l_hausdorff
    df_hold_l['Hold mIoU'] = hold_l_mIoU
    df_external_l['External mIoU'] = external_l_mIoU
    df_hold_l['Hold ROC'] = hold_l_ROC
    df_hold_l['Hold PR'] = hold_l_PR
    df_external_l['External ROC'] = external_l_ROC
    df_external_l['External PR'] = external_l_PR
    df_hold_l.to_csv(f'{save_path}_hold_l.csv')
    df_external_l.to_csv(f'{save_path}_external_l.csv')

In [12]:
hold_path = glob.glob(f'{root}/hold_processed/*/*')
external_path = glob.glob(f'{root}/external_processed/*/*')
model_path = f'{root}/models_raw/{name}.hdf5'
save_path = f'{root}/models_raw/{name}'

hold = load_dataset(hold_path)
external = load_dataset(external_path)

model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)

vessel_lumen()
vessel()
lumen()

1/1 [==============================] - 0s 94ms/step


/home/jjlou/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


1/1 [==============================] - 0s 89ms/step
